# Iterative Proportional Fitting Code Demo


Code from https://github.com/Dirguis/ipfn (IPFN package for Python).

In [5]:
from ipfn import ipfn
import numpy as np
import pandas as pd
census_data_pd = pd.read_excel('planning_database_NYC_tracts.xlsx')

In [6]:
df = census_data_pd
df['County_name'].drop_duplicates()

0          Bronx County
339        Kings County
1100    New York County
1388      Queens County
2057    Richmond County
Name: County_name, dtype: object

In [25]:
df_nyc = df[df['County_name'].str.match('New York County')]
df_msh = df[df['Neighborhood'].str.match('Morning')]

In [30]:
df_msh_tot = df_msh.loc[:, df.columns.str.startswith('Tot_Population_CEN_2010')]
df_msh_tot # total = 38723

,Tot_Population_CEN_2010
1303,641.0
1306,10064.0
1308,1731.0
1310,3633.0
1311,5322.0
1313,3329.0
1315,3673.0
1317,10330.0


In [105]:
df_msh_rent = df_msh.loc[:, df.columns.str.startswith('pct_Renter_Occp_HU_CEN_2010')]
df_msh_rent = df_msh_rent['pct_Renter_Occp_HU_CEN_2010'].apply(lambda num : num/100.0) 
msh_rent = np.column_stack([df_msh_rent, 1-df_msh_rent])
msh_rent

array([[1.    , 0.    ],
       [0.7345, 0.2655],
       [0.8533, 0.1467],
       [0.9639, 0.0361],
       [0.8169, 0.1831],
       [0.9584, 0.0416],
       [0.9526, 0.0474],
       [0.74  , 0.26  ]])

In [35]:
df_msh_race = df_msh[['Hispanic_CEN_2010','NH_White_alone_CEN_2010','NH_Blk_alone_CEN_2010','NH_AIAN_alone_CEN_2010','NH_Asian_alone_CEN_2010','NH_NHOPI_alone_CEN_2010','NH_SOR_alone_CEN_2010']]
df_msh_race

,Hispanic_CEN_2010,NH_White_alone_CEN_2010,NH_Blk_alone_CEN_2010,NH_AIAN_alone_CEN_2010,NH_Asian_alone_CEN_2010,NH_NHOPI_alone_CEN_2010,NH_SOR_alone_CEN_2010
1303,90.0,331.0,102.0,2.0,100.0,0.0,4.0
1306,1315.0,6001.0,655.0,12.0,1692.0,7.0,25.0
1308,211.0,907.0,143.0,4.0,382.0,2.0,7.0
1310,512.0,1713.0,354.0,6.0,877.0,3.0,9.0
1311,529.0,3428.0,347.0,11.0,798.0,4.0,32.0
1313,342.0,2015.0,181.0,4.0,659.0,3.0,15.0
1315,1632.0,164.0,1664.0,4.0,127.0,1.0,13.0
1317,2734.0,3829.0,2099.0,35.0,1334.0,2.0,36.0


In [127]:
m = np.ones( (len(df_msh),len(df_msh_race.columns),2) )
m.shape # track, race, rent

(8, 7, 2)

In [128]:
xipp = np.array(df_msh_tot)
xijp = np.array(df_msh_race)
xipk = np.array(msh_rent)
xpjp = xijp.sum(axis=0)
xppk = xpjk.sum(axis=0)

In [129]:
aggregates = [xipp, xpjp, xppk, xijp, xpjk]
dimensions = [[0], [1], [2], [0, 1], [1, 2]]

In [130]:
IPF = ipfn.ipfn(m, aggregates, dimensions)
m = IPF.iteration()
print(xijp[0,0])
print(m[0, 0, :].sum())

ipfn converged: convergence_rate not updating or below rate_tolerance
90.0
0.012219959266802444


/usr/local/lib/python3.7/site-packages/ipfn/ipfn.py:129: RuntimeWarning: invalid value encountered in double_scalars
  if abs(m_ijk / ori_ijk - 1) > max_conv:
